# Dockerfile instructions

Here are the most basic options for the dockerfile.

- `FROM` - sets the base image for the new image;
- `WORKDIR` - sets the working directory;
- `COPY <host directory> <image directory>` - copy file from host to the image;
- `RUN` - allows to run command during container build. 

# Run with start of container

The instructions provided in this section allow you to set container behaviour with the command specified when starting the container in `docker run`. So [here](docker_commands.ipynb#sec-docker_run_com_on_start) you can find some useful tips.

## `ENTRYPOINT`

Allow command to run when container based on image is running. The feature of this instruction is that you can add something at the end of the `docker run` command, and it will be added to command described in `ENTRYPOINT`.

Such dockerfile:
```
FROM ubuntu:20.04
ENTRYPOINT [ "echo" ]
```

Creates a container that will `echo` the line declared at the end of the docker run command. For exmaple:

In [5]:
%%bash
docker build \
    -f basic_instructions/entrypoint_example \
    -t test_ubuntu \
    basic_instructions &> /dev/null

docker run --rm test_ubuntu "hello world"
docker run --rm test_ubuntu ls
docker rmi test_ubuntu &> /dev/null

hello world
ls


So the lines I passed as arguments were just printed - just like `echo` does.

## `CMD`

Allow command to run when container based on image is running. If using this instruction you set any command for `docker run`, the command described in `CMD` will be completely removed with new commad.

Consider containers based on dockerfile:
```
FROM ubuntu:20.04
CMD [ "echo" ]
```

The same command `docker run --rm test_ubuntu ls` like it was for the dockerfile from `ENTRYPOINT` example, desn't print "ls" but executes the `ls` command.

In [8]:
%%bash
docker build \
    -f basic_instructions/cmd_example \
    -t test_ubuntu \
    basic_instructions &> /dev/null
docker run --rm test_ubuntu ls 
docker rmi test_ubuntu &> /dev/null

bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var


## `ENTRYPOINT` + `CMD`

By combining `ENTRYPOINT` and `CMD` you can do this:
- in `ENTRYPOIN` set the necessary part of the command;
- in `CMD` set the part of the command that can be changed.

docker file like:

```
FROM ubuntu:20.04
ENTRYPOINT ["echo"]
CMD ["default message"]
```

By default, "default message" is printed, but if you add a line to `docker run' it will be printed instead. The following cell show how it works:

In [9]:
%%bash
docker build \
    -f basic_instructions/entrypoint_cmd \
    -t test_ubuntu \
    basic_instructions &> /dev/null
docker run --rm test_ubuntu
docker run --rm test_ubuntu "other message"
docker rmi test_ubuntu &> /dev/null

default message
other message


Consequence of instractions doesn't matter. So docker file:
```
FROM ubuntu:20.04
CMD ["default message"]
ENTRYPOINT ["echo"]
```
Will work the same way. The folowing cell show this scenario.

In [10]:
%%bash
docker build \
    -f basic_instructions/entrypoint_cmd2 \
    -t test_ubuntu \
    basic_instructions &> /dev/null
docker run --rm test_ubuntu
docker run --rm test_ubuntu "other message"
docker rmi test_ubuntu &> /dev/null

default message
other message


#### Several `CMD`/`ENTRYPOIN`

Looks like, docker only takes the last mention of `CMD`/`ENTRY POINT`. Which makes sense in the case of 'inheriting' containers. So at the following example I'm using docker file:

```
FROM ubuntu:20.04
CMD ["echo", "default message1"]
CMD ["echo", "default message2"]
```

In [11]:
%%bash
docker build \
    -f basic_instructions/several_cmd \
    -t test_ubuntu \
    basic_instructions &> /dev/null
docker run --rm test_ubuntu
docker rmi test_ubuntu &> /dev/null

default message2


So it looks like docker just ignored `CMD ["echo", "default message1"]`

# Environment variables

For creating environment variable you can use `ENV` dockerfile instruction. 

In following example I create a dockefile vich use `ENV` to create `TEST_VAR` with value `"test variable value"`. Then print it from container.

In [22]:
%%bash
cd basic_instructions

echo "FROM ubuntu" > envdockerfile
echo "ENV TEST_VAR=\"test variable value\"" >> envdockerfile

docker build -t test_image -f envdockerfile . &> /dev/null
docker run --rm -itd --name test_container test_image &> /dev/null
docker exec test_container bash -c "echo \$TEST_VAR"


docker stop test_container &> /dev/null
docker rmi test_image &> /dev/null
rm envdockerfile

test variable value
